In [1]:
# Import libraries
import numpy as np
import pandas as pd
from matplotlib.pyplot import plot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
df = pd.read_csv('NGSIM_US101.csv')

In [3]:
df.head()

,Unnamed: 0,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway,Congestion
0,0,515,2330.0,1123.0,1.118848e+12,30.034,188.062,6451203.729,1873252.549,13.0,6.9,2.0,23.31,2.05,3.0,500.0,523.0,119.10,5.11,0
1,1,515,2330.0,1123.0,1.118848e+12,30.034,188.062,6451203.729,1873252.549,13.0,6.9,2.0,23.31,2.05,3.0,500.0,523.0,119.10,5.11,0
2,3,2127,6459.0,567.0,1.118848e+12,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,2.0,37.52,11.20,2.0,2124.0,2132.0,48.92,1.30,0
3,4,1033,4827.0,592.0,1.118848e+12,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,2.0,41.99,0.10,1.0,1029.0,1040.0,38.81,0.92,0
4,5,1033,4827.0,592.0,1.118848e+12,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,2.0,41.99,0.10,1.0,1029.0,1040.0,38.81,0.92,0


In [4]:
df.columns

Index(['Unnamed: 0', 'Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Global_Time',
       'Local_X', 'Local_Y', 'Global_X', 'Global_Y', 'v_length', 'v_Width',
       'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding', 'Following',
       'Space_Headway', 'Time_Headway', 'Congestion'],
      dtype='object')

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.duplicated().sum()

72064

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455846 entries, 0 to 527909
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Vehicle_ID     455846 non-null  int64  
 1   Frame_ID       455846 non-null  float64
 2   Total_Frames   455846 non-null  float64
 3   Global_Time    455846 non-null  float64
 4   Local_X        455846 non-null  float64
 5   Local_Y        455846 non-null  float64
 6   Global_X       455846 non-null  float64
 7   Global_Y       455846 non-null  float64
 8   v_length       455846 non-null  float64
 9   v_Width        455846 non-null  float64
 10  v_Class        455846 non-null  float64
 11  v_Vel          455846 non-null  float64
 12  v_Acc          455846 non-null  float64
 13  Lane_ID        455846 non-null  float64
 14  Preceding      455846 non-null  float64
 15  Following      455846 non-null  float64
 16  Space_Headway  455846 non-null  float64
 17  Time_Headway   455846 non-nul

# Handling data columns

Most of the columns contain data: 
- are not necessary for prediction  
- would create a bias in our prediction model
- would make it impossible to use the model in real time

In [9]:
df.drop(['Vehicle_ID', 'Frame_ID', 'Total_Frames',
        'Preceding', 'Following', 'Space_Headway', 'Time_Headway'], axis=1, inplace=True)

In [10]:
df.head()

,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Congestion
0,1.118848e+12,30.034,188.062,6451203.729,1873252.549,13.0,6.9,2.0,23.31,2.05,3.0,0
2,1.118848e+12,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,2.0,37.52,11.20,2.0,0
3,1.118848e+12,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,2.0,41.99,0.10,1.0,0
5,1.118850e+12,53.514,817.521,6451655.238,1872800.663,24.0,8.5,3.0,45.12,-0.55,5.0,0
6,1.118848e+12,28.878,490.086,6451422.353,1873041.018,15.0,5.9,2.0,24.54,-0.04,3.0,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455846 entries, 0 to 527909
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Global_Time  455846 non-null  float64
 1   Local_X      455846 non-null  float64
 2   Local_Y      455846 non-null  float64
 3   Global_X     455846 non-null  float64
 4   Global_Y     455846 non-null  float64
 5   v_length     455846 non-null  float64
 6   v_Width      455846 non-null  float64
 7   v_Class      455846 non-null  float64
 8   v_Vel        455846 non-null  float64
 9   v_Acc        455846 non-null  float64
 10  Lane_ID      455846 non-null  float64
 11  Congestion   455846 non-null  int64  
dtypes: float64(11), int64(1)
memory usage: 45.2 MB


In [12]:
# Convert data types
df['Global_Time'] = pd.to_datetime(df['Global_Time'], unit='ms').dt.tz_localize('GMT').dt.tz_convert('US/Pacific')

In [13]:
df.head()

,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Congestion
0,2005-06-15 08:07:55-07:00,30.034,188.062,6451203.729,1873252.549,13.0,6.9,2.0,23.31,2.05,3.0,0
2,2005-06-15 08:00:24.800000-07:00,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,2.0,37.52,11.20,2.0,0
3,2005-06-15 08:12:04.700000-07:00,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,2.0,41.99,0.10,1.0,0
5,2005-06-15 08:34:32.700000-07:00,53.514,817.521,6451655.238,1872800.663,24.0,8.5,3.0,45.12,-0.55,5.0,0
6,2005-06-15 08:09:41.200000-07:00,28.878,490.086,6451422.353,1873041.018,15.0,5.9,2.0,24.54,-0.04,3.0,1


In [15]:
df['Time_Hour'] = df['Global_Time'].dt.hour
df['Time_Minute'] = df['Global_Time'].dt.minute
df['Time_Second'] = df['Global_Time'].dt.second

In [16]:
df.head()

,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Congestion,Time_Hour,Time_Minute,Time_Second
0,2005-06-15 08:07:55-07:00,30.034,188.062,6451203.729,1873252.549,13.0,6.9,2.0,23.31,2.05,3.0,0,8,7,55
2,2005-06-15 08:00:24.800000-07:00,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,2.0,37.52,11.20,2.0,0,8,0,24
3,2005-06-15 08:12:04.700000-07:00,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,2.0,41.99,0.10,1.0,0,8,12,4
5,2005-06-15 08:34:32.700000-07:00,53.514,817.521,6451655.238,1872800.663,24.0,8.5,3.0,45.12,-0.55,5.0,0,8,34,32
6,2005-06-15 08:09:41.200000-07:00,28.878,490.086,6451422.353,1873041.018,15.0,5.9,2.0,24.54,-0.04,3.0,1,8,9,41
